In [1]:
import json 
import pandas as pd 
import soft_kmeans_alg 
import new_utils 
import json 
from scipy.stats import kendalltau

In [2]:
n_shuffle = 2
iterations = 100
burn_in = 10
thining = 2

In [3]:
data = pd.read_csv("data/synthetic/5|50_2.csv")
data.head()
n_biomarkers = len(data.biomarker.unique())
# biomarker_df = data[data['biomarker'] == "HIP-FCI (1)"].reset_index(drop=True)
# biomarker_df.head()

In [4]:
all_current_accepted_order_dicts = soft_kmeans_alg.metropolis_hastings_soft_kmeans(
    data,
    iterations,
    n_shuffle,
)

iteration 10 done, current accepted likelihood: -357.2684627993102, current acceptance ratio is 40.00 %, current accepted order is {'HIP-FCI (1)': 4, 'PCC-FCI (2)': 2, 'HIP-GMI (3)': 1, 'FUS-GMI (4)': 3, 'FUS-FCI (5)': 5}, 
iteration 20 done, current accepted likelihood: -332.3792331191731, current acceptance ratio is 35.00 %, current accepted order is {'HIP-FCI (1)': 1, 'PCC-FCI (2)': 2, 'HIP-GMI (3)': 4, 'FUS-GMI (4)': 3, 'FUS-FCI (5)': 5}, 
iteration 30 done, current accepted likelihood: -332.3792331191731, current acceptance ratio is 23.33 %, current accepted order is {'HIP-FCI (1)': 1, 'PCC-FCI (2)': 2, 'HIP-GMI (3)': 4, 'FUS-GMI (4)': 3, 'FUS-FCI (5)': 5}, 
iteration 40 done, current accepted likelihood: -332.1403343275587, current acceptance ratio is 30.00 %, current accepted order is {'HIP-FCI (1)': 1, 'PCC-FCI (2)': 2, 'HIP-GMI (3)': 4, 'FUS-GMI (4)': 3, 'FUS-FCI (5)': 5}, 
iteration 50 done, current accepted likelihood: -332.1403343275587, current acceptance ratio is 24.00 %,

In [7]:
new_utils.save_heatmap(all_current_accepted_order_dicts,
                       burn_in, thining, folder_name='img',
                       file_name='test_heatmap', title='heatmap')

In [8]:
most_likely_order_dic = new_utils.obtain_most_likely_order_dic(
    all_current_accepted_order_dicts, burn_in, thining)
most_likely_order = list(most_likely_order_dic.values())
tau, p_value = kendalltau(most_likely_order, range(1, n_biomarkers + 1))
dic = {}
dic['5|50'] = []
dic['5|50'].append(tau)
# write the JSON to a file
with open("results.json", "w") as file:
    json.dump(dic, file, indent=4)

In [1]:
# new_utils.get_theta_phi_estimates(data)

In [5]:
# biomarker_df.measurement.values

In [6]:
# # reshape to satisfy sklearn requirements
# measurements = np.array(biomarker_df['measurement']).reshape(-1, 1)

# # dataframe for non-diseased participants
# healthy_df = biomarker_df[biomarker_df['diseased'] == False]


In [7]:
# kmeans_setup = KMeans(2, random_state=0, n_init="auto")
# hierarchical_clustering_setup = AgglomerativeClustering(n_clusters=2)

In [8]:
# clustering_result_kmeans = kmeans_setup.fit(measurements)
# clustering_result_kmeans.labels_

In [9]:
# clustering_result_kmeans.predict(measurements)

In [10]:
# theta_mean, theta_std, phi_mean, phi_std = utils.compute_theta_phi_for_biomarker(
#     data, "HIP-FCI (1)", clustering_setup = None)

In [11]:
# theta_mean, theta_std, phi_mean, phi_std

In [12]:
# biomarkers = data.biomarker.unique()
# utils.get_theta_phi_estimates(data, biomarkers)